<a id='sect0'></a>
## <b><font color='darkblue'>Meet ChromaDB for LLM Applications</font><b/>
<b><font size='3ptx'>[ChromaDB](https://docs.trychroma.com/) is an open-source vector database designed specifically for LLM applications.</font></b>
* <b><font size='3ptx'><a href='#sect0_1'>Store documents</font></b>
* <b><font size='3ptx'><a href='#sect0_2'>Query Vectorestore</font></b>
* <b><font size='3ptx'><a href='#sect0_3'>Update documents</font></b>
* <b><font size='3ptx'><a href='#sect0_4'>Delete documents</font></b>

<b>ChromaDB offers you both a user-friendly API and impressive performance, making it a great choice for many embedding applications</b>. To get started, activate your virtual environment and run the following command:
```shell
(venv) $ python -m pip install chromadb
```

<br/>

If you have any issues installing ChromaDB, take a look at [the troubleshooting guide](https://docs.trychroma.com/troubleshooting#build-error-when-running-pip-install-chromadb) for help.

<a id='sect0_1'></a>
### <b><font color='darkgreen'>Store documents</font></b> ([back](#sect0))
<font size='3ptx'><b>Because you have a grasp on vectors and embeddings, and you understand the motivation behind vector databases, the best way to get started is with an example</b></font>.  

<b>For this example, you’ll store ten documents to search over.</b> To illustrate the power of embeddings and semantic search, each document covers a different topic, and you’ll see how well ChromaDB associates your queries with similar documents.

In [1]:
# from .autonotebook import tqdm as notebook_tqdm
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "demo_docs"

client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [2]:
collections = client.list_collections()

In [3]:
collections

[Collection(name=demo_docs)]

Next, you instantiate your embedding function and the ChromaDB collection to store your documents in:

In [4]:
has_registered = False
func_select_embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction

if not collections:
    embedding_func = func_select_embedding_model(
        model_name=EMBED_MODEL
    )

    collection = client.create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_func,
        metadata={"hnsw:space": "cosine"},
    )
else:
    has_registered = True
    collection = collections[0]

print(f'has_registered is {has_registered}')

has_registered is True


You specify an embedding function from the [**SentenceTransformers**](https://sbert.net/) library. ChromaDB will use this to embed all your documents and queries. In this example, you’ll continue using the "`all-MiniLM-L6-v2`" model. You then create your first collection.

<b>A collection is the object that stores your embedded documents along with any associated metadata</b>. If you’re familiar with relational databases, then you can think of a collection as a table. In this example, your collection is named demo_docs, it uses the "`all-MiniLM-L6-v2`" embedding function that you instantiated, and it uses the cosine similarity distance function as specified by `metadata={"hnsw:space": "cosine"}`.

The last step in setting up your collection is to add documents and metadata:

In [5]:
documents = [
     "The latest iPhone model comes with impressive features and a powerful camera.",
     "Exploring the beautiful beaches and vibrant culture of Bali is a dream for many travelers.",
     "Einstein's theory of relativity revolutionized our understanding of space and time.",
     "Traditional Italian pizza is famous for its thin crust, fresh ingredients, and wood-fired ovens.",
     "The American Revolution had a profound impact on the birth of the United States as a nation.",
     "Regular exercise and a balanced diet are essential for maintaining good physical health.",
     "Leonardo da Vinci's Mona Lisa is considered one of the most iconic paintings in art history.",
     "Climate change poses a significant threat to the planet's ecosystems and biodiversity.",
     "Startup companies often face challenges in securing funding and scaling their operations.",
     "Beethoven's Symphony No. 9 is celebrated for its powerful choral finale, 'Ode to Joy.'",
]

genres = [
     "technology",
     "travel",
     "science",
     "food",
     "history",
     "fitness",
     "art",
     "climate change",
     "business",
     "music",
]

In [6]:
if not has_registered:
    collection.add(
        documents=documents,
         ids=[f"id{i}" for i in range(len(documents))],
         metadatas=[{"genre": g} for g in genres]
    )

<b>The `metadatas` argument is optional, but most of the time, it’s useful to store metadata with your embeddings. In this case, you define a single metadata field, "genre", that records the genre of each document</b>. When you query a document, metadata provides you with additional information that can be helpful to better understand the document’s contents. You can also filter on metadata fields, just like you would in a relational database query.

<a id='sect0_2'></a>
### <b><font color='darkgreen'>Query Vectorestore</font></b> ([back](#sect0))
<b><font size='3ptx'>With documents embedded and stored in a collection, you’re ready to run some semantic queries.</font></b>

Below code snippet will send query `Find me some delicious food!` and request only one doc being returned:

In [7]:
query_results = collection.query(
    query_texts=["Find me some delicious food!"],
    n_results=1)

Update of nonexisting embedding ID: id1
Update of nonexisting embedding ID: id2
Delete of nonexisting embedding ID: id1
Delete of nonexisting embedding ID: id2
Update of nonexisting embedding ID: id1
Update of nonexisting embedding ID: id2
Delete of nonexisting embedding ID: id1
Delete of nonexisting embedding ID: id2


In [8]:
query_results.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

In [9]:
query_results["documents"]

[['Traditional Italian pizza is famous for its thin crust, fresh ingredients, and wood-fired ovens.']]

In [10]:
query_results["ids"]

[['id3']]

In [11]:
query_results["distances"]

[[0.7638262063498773]]

In [12]:
query_results["metadatas"]

[[{'genre': 'food'}]]

As you can see, the embedding for `Traditional Italian pizza is famous for its thin crust, fresh ingredients, and wood-fired ovens` was most similar to the query `Find me some delicious food`. You probably agree that this document is the closest match. You can also see the ID, metadata, and distance associated with the matching document embedding. Here, you’re using **cosine distance**, which is one minus the cosine similarity between two embeddings.

With <font color='blue'>collection.query()</font>, you’re not limited to single queries or single results:

In [13]:
query_results = collection.query(
    query_texts=["Teach me about history",
                 "What's going on in the world?"],
    include=["documents", "distances"],
    n_results=2)

In [14]:
query_results["documents"][0]

['The American Revolution had a profound impact on the birth of the United States as a nation.',
 "Leonardo da Vinci's Mona Lisa is considered one of the most iconic paintings in art history."]

In [15]:
query_results["distances"][0]

[0.6904192480258038, 0.8771601240607931]

In [16]:
query_results["documents"][1]

["Climate change poses a significant threat to the planet's ecosystems and biodiversity.",
 'The American Revolution had a profound impact on the birth of the United States as a nation.']

In [17]:
query_results["distances"][1]

[0.8002942768712199, 0.9402920899385823]

For this query, the two most similar documents weren’t as strong of a match as in the first query. Recall that cosine distance is one minus cosine similarity, so a cosine distance of 0.80 corresponds to a cosine similarity of 0.20.

<b><font color='darkred'>Note:</font></b>
> <b>Keep in mind that so-called similar documents returned from a semantic search over embeddings may not actually be relevant to the task that you’re trying to solve</b>. The success of a semantic search is somewhat subjective, and you or your stakeholders might not agree on the quality of the results.
> <br/><br/>
> <b>If there are no relevant documents in your collection for a given query, or your embedding algorithm wasn’t trained on the right or enough data, then your results might be poor</b>. It’s up to you to understand your application, your stakeholders’ expectations, and the limitations of your embedding algorithm and document collection.

<b>Another awesome feature of ChromaDB is the ability to filter queries on metadata</b>. To motivate this, suppose you want to find the single document that’s most related to music history. You might run this query:

In [18]:
collection.query(
    query_texts=["Teach me about music history"],
    n_results=1)

{'ids': [['id9']],
 'distances': [[0.8186328079302339]],
 'metadatas': [[{'genre': 'music'}]],
 'embeddings': None,
 'documents': [["Beethoven's Symphony No. 9 is celebrated for its powerful choral finale, 'Ode to Joy.'"]],
 'uris': None,
 'data': None}

our query is `Teach me about music history`, and the most similar document is `Einstein’s theory of relativity revolutionized our understanding of space and time`. While Einstein is a historical figure who was a musician and teacher, this isn’t quite the result that you’re looking for. Because you’re particularly interested in `music` history, you can filter on the "genre" metadata field to search over more relevant documents:

In [19]:
collection.query(
    query_texts=["Teach me about music history"],
    where={"genre": {"$eq": "music"}},
    n_results=1)

{'ids': [['id9']],
 'distances': [[0.8186328079302339]],
 'metadatas': [[{'genre': 'music'}]],
 'embeddings': None,
 'documents': [["Beethoven's Symphony No. 9 is celebrated for its powerful choral finale, 'Ode to Joy.'"]],
 'uris': None,
 'data': None}

As you can see, the document about `Beethoven’s Symphony No. 9` is the most similar document. Of course, for this example, there’s only one document with the `music` genre. To make it slightly more difficult, you could filter on both `history` and `music`:

In [20]:
query_results = collection.query(
    query_texts=["Teach me about music history"],
    where={"genre": {"$in": ["music", "history"]}},
    n_results=2,
)

In [21]:
query_results["documents"]

[["Beethoven's Symphony No. 9 is celebrated for its powerful choral finale, 'Ode to Joy.'",
  'The American Revolution had a profound impact on the birth of the United States as a nation.']]

In [22]:
query_results["distances"]

[[0.8186328079302339, 0.8200413485985653]]

This query filters the collection of documents that have either a music or history genre, as specified by `where={"genre": {"$in": ["music", "history"]}}`. As you can see, the `Beethoven document` is still the most similar, while the `American Revolution document` is a close second. These were straightforward filtering examples on a single metadata field, but ChromaDB also supports [**other filtering operations**](https://docs.trychroma.com/usage-guide#:~:text=Filtering%20metadata%20supports%20the%20following%20operators%3A) that you might need.

<a id='sect0_3'></a>
### <b><font color='darkgreen'>Update documents</font></b> ([back](#sect0))
<font size='3ptx'><b>If you want to update existing documents, embeddings, or metadata, then you can use <font color='blue'>collection.update()</font>.</b></font>


This requires you to know the IDs of the data that you want to update. In this example, you’ll update both the documents and metadata for "id1" and "id2":

In [23]:
collection.update(
    ids=["id1", "id2"],
    documents=[
        "The new iPhone is awesome!",
        "Bali has beautiful beaches"],
    metadatas=[{"genre": "tech"}, {"genre": "beaches"}]
)

Update of nonexisting embedding ID: id1
Update of nonexisting embedding ID: id2
Update of nonexisting embedding ID: id1
Update of nonexisting embedding ID: id2


In [24]:
query_results = collection.get(ids=["id1", "id2"])

In [25]:
query_results["documents"]

[]

In [26]:
query_results["metadatas"]

[]

<a id='sect0_4'></a>
### <b><font color='darkgreen'>Delete documents</font></b> ([back](#sect0))
<font size='3ptx'><b>Lastly, if you want to delete any items in the collection, then you can use <font color='blue'>collection.delete()</font>.</b></font>

In [27]:
print(f'Before deletion, we have {collection.count()} document(s)!')

Before deletion, we have 8 document(s)!


Below code snippet will delete two documents with id `id1` and `id2`:

In [28]:
collection.delete(ids=["id1", "id2"])

Delete of nonexisting embedding ID: id1
Delete of nonexisting embedding ID: id2
Delete of nonexisting embedding ID: id1
Delete of nonexisting embedding ID: id2


In [29]:
collection.get(["id1", "id2"])

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [30]:
print(f'After deletion, we have {collection.count()} document(s)!')

After deletion, we have 8 document(s)!


You’ve now seen many of ChromaDB’s main features, and you can learn more with the [**getting started guide**](https://docs.trychroma.com/getting-started) or [**API cheat sheet**](https://docs.trychroma.com/api-reference). You used a collection of ten hand-crafted documents that allowed you to get familiar with ChromaDB’s syntax and querying functionality, but this was by no means a realistic use case. <b>In the next section, you’ll see ChromaDB shine while you embed and query over thousands of real-world documents</b>!

<a id='sect1'></a>
## <b><font color='darkblue'>Practical Example: Add Context for a Large Language Model (LLM)</font></b>
<b><font size='3ptx'>Vector databases are capable of storing all types of embeddings, such as text, audio, and images. However, as you’ve learned, ChromaDB was initially designed with text embeddings in mind, and it’s most often used to build LLM applications.</font></b>
* <b><font size='3ptx'><a href='#sect1_1'>Prepare and Inspect Your Dataset</a></font></b>
* <b><font size='3ptx'><a href='#sect1_2'>Create a Collection and Add Reviews</a></font></b>
* <b><font size='3ptx'><a href='#sect1_3'>Connect to an LLM Service</a></font></b>
* <b><font size='3ptx'><a href='#sect1_4'>Provide Context to the LLM</a></font></b>

<b>In this section, you’ll get hands-on experience using ChromaDB to provide context to OpenAI’s ChatGPT LLM</b>. To set the scene, you’re a software engineer who works on a popular repo **["bt_test_common"](https://github.com/johnklee/bt_test_common)** (Common utilities for BT testing.). You want to help external users to learn more about this repo and how to use the APIs provided by this repo by LLM.

You’re responsible for designing and implementing the back-end logic that creates these summaries. You’ll take the following steps:
1. <b>Create a ChromaDB collection that stores documents</b> along with associated metadata.
2. <b>Create a system that accepts a query, finds semantically similar documents</b>, and uses the similar documents as context to an LLM. The LLM will use the documents to answer the question posed in the query.

<b>This process of retrieving relevant documents and using them as context for a generative model is known as [retrieval-augmented generation](https://cloud.google.com/use-cases/retrieval-augmented-generation?hl=en) (RAG)</b>. This allows LLMs to make inferences using information that wasn’t included in their training dataset, and this is the most common way to apply ChromaDB in LLM applications.

There are lots of factors and variations to consider when implementing a RAG system, but for this example, you’ll only need to know the fundamentals. Here’s what a RAG system might look like with ChromaDB:

![RAG](images/rag_diagram.PNG)

We first embed and store the documents in a ChromaDB collection. In this example, those documents are coming from the `bttc` repo. We then run a query  through ChromaDB to find semantically relevant documents, and you pass the query and relevant documents to an LLM to generate a context-informed response.

<b>The key here is that the LLM takes both the original query and the relevant documents as input, allowing it to generate a meaningful response that it wouldn’t be able to create without the documents.</b>

In reality, your deliverable for this project would likely be a [**chatbot**](https://realpython.com/build-a-chatbot-python-chatterbot/) that stakeholders use to ask questions about repo `bttc` through a user interface. While building a full-fledged chatbot is beyond the scope of this tutorial, you can check out libraries like [**LangChain**](https://python.langchain.com/docs/get_started/introduction) that are designed specifically to help you assemble LLM applications.

**The focus of this example is for you to see how you can use ChromaDB for RAG**. This practical knowledge will help reduce the learning curve for [**LangChain**](https://python.langchain.com/docs/get_started/introduction) if you choose to go that route in the future. With that, you’re ready to get started!

<a id='sect1_1'></a>
### <b><font color='darkgreen'>Prepare and Inspect Your Dataset</font></b> ([back](#sect1))
<b><font size='3ptx'>You’ll use the repo of [`bttc`](https://github.com/johnklee/bt_test_common) to create the collection. </font></b>

Once you’ve clone the target repo [`bttc`](https://github.com/johnklee/bt_test_common), export the root path of repo as environment variable `BTTC_REPO_ROOT` for future ingestion:
```shell
$ git clone git@github.com:johnklee/bt_test_common.git
$ cd bt_test_common/
$ export BTTC_REPO_ROOT=`pwd`
$ env | grep BTTC_REPO_ROOT
BTTC_REPO_ROOT=/usr/local/google/home/johnkclee/Github/bt_test_common
```

Here’s a function that you can use to create the dataset of target repo `bttc` for ingestion of ChromaDB:

In [1]:
import ingest_bttc_repo

BTTC_REPO_ROOT='/usr/local/google/home/johnkclee/Github/bt_test_common'

In [2]:
for dataset in ingest_bttc_repo.ingestion_of_repo_dataset_gen(BTTC_REPO_ROOT):
    ids = dataset['ids']
    documents = dataset['documents']
    metadatas = dataset['metadatas']
    print(f'ids: {ids}')
    print(f'Sample doc:')
    for doc, metadata in zip(documents, metadatas):
        print(f'\tmetadata: {metadata}')
        print(f'\t{doc[:20]} ({len(doc):,d} chs)')
        break
    break

ids: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
Sample doc:
	metadata: {'file_path': '/usr/local/google/home/johnkclee/Github/bt_test_common/setup.py', 'file_type': 'py'}
	from __future__ impo (1,772 chs)


Function <font color='blue'><b>ingest_bttc_repo</b>.ingestion_of_repo_dataset_gen</font> will generate dataset for every 10 documents for ingestion.

In [3]:
for dataset in ingest_bttc_repo.ingestion_of_repo_dataset_gen_with_split(BTTC_REPO_ROOT):
    ids = dataset['ids']
    documents = dataset['documents']
    metadatas = dataset['metadatas']
    print(f'ids: {ids}')
    print(f'Sample doc:')
    for doc, metadata in zip(documents, metadatas):
        print(f'\tmetadata: {metadata}')
        print(f'\t{doc[:20]} ({len(doc):,d} chs)')
        break
    break

ids: ['2', '3']
Sample doc:
	metadata: {'file_path': '/usr/local/google/home/johnkclee/Github/bt_test_common/setup.py', 'file_type': 'py'}
	from __future__ impo (1,518 chs)


<a id='sect1_2'></a>
### <b><font color='darkgreen'>Create a Collection and Add Reviews</font></b> ([back](#sect1))
<b><font size='3ptx'>Next, you’ll create a collection and add the reviews.</font></b>

Below codesnippet will ingest the documents for further RAG usage:

In [4]:
CHROMA_PATH = "bttc_repo_embeddings"
EMBEDDING_FUNC_NAME = "multi-qa-MiniLM-L6-cos-v1"
COLLECTION_NAME = "bttc_docs"

In [5]:
%%time
collection = ingest_bttc_repo.build_chroma_collection(
    chroma_path=CHROMA_PATH,
    collection_name=COLLECTION_NAME,
    embedding_func_name=EMBEDDING_FUNC_NAME,
    default_repo_path=BTTC_REPO_ROOT)

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
In

CPU times: user 5min 19s, sys: 1min 14s, total: 6min 33s
Wall time: 23.4 s


In [7]:
print(f'Total {collection.count()} document(s) being ingested!')

Total 292 document(s) being ingested!


Now we could query the created collection:

In [8]:
query_result = collection.query(
    query_texts=["Tell me how to turn on the bluetooth."],
    n_results=5,
    include=["documents", "distances", "metadatas"])

In [9]:
len(query_result['documents'][0][0])

912

In [10]:
len(query_result['documents'][0][0].split('\n'))

28

In [11]:
# Get the first querying result for first document
print(query_result['documents'][0][0][500:1000])

l', value='null', err='')

# Turn on the Gabeldorsche (GD) verbose log
>>> dut.bt.enable_gd_log_verbose()
True

# Confirm the setting is set successfully (value='true')
>>> dut.gm.device_config.bluetooth['INIT_logging_debug_enabled_for_all']
DeviceConfig.Namespace.Setting(name='INIT_logging_debug_enabled_for_all', value='true', err='')
```

Ps. For this setting to work, your `build_version_sdk` should >= 33.



<a id='sect1_3'></a>
### <b><font color='darkgreen'>Connect to an LLM Service</font></b> ([back](#sect1))
<b><font size='3ptx'>As you know, you’re going to use the `bttc` embeddings as context to an LLM.</font></b>

This means that you’ll ask the LLM a question like `How to turn off the bluebooth`, and you’ll provide relevant embeddings to help the LLM answer this question. To do this, you’ll first need to install the [**openai**](https://github.com/openai/openai-python) library:
```shell
(venv) $ python -m pip install openai
```

<b>You need an API key to interact with the models in the openai library, and you can check out [this tutorial](https://realpython.com/generate-images-with-dalle-openai-api/#get-your-openai-api-key) to help you get set up</b>. Once you have your API key, you can store it as an environment variable or add it to a configuration file like this [**JSON**](https://realpython.com/python-json/) file that you could name <font color='olive'>config.json</font>:
```json
{
    "openai-secret-key": "<your-api-key>"
}
```

To make sure your API works and everything is running properly, you can run the following code, which will ask the LLM a question without considering any of the documents in your ChromaDB collection:

In [12]:
import os
import json
import llm_utils
import openai

# 0) Initialize an LLM agent.
llm_agent = llm_utils.LLMAgent(
    context='You are a software engineer.',  # Context for the role of LLM agent.
    model_name='gpt-3.5-turbo'               # Model to use.
)

In [13]:
def print_answer(answer: str, max_line_width: int=100):
    lines = [line.strip() for line in answer.split('\n')]
    for line in lines:
        if not line:
            print('')
            continue

        text = ''
        text_length = 0
        for word in line.split(' '):
            word_length = len(word)
            if text_length + word_length <= max_line_width:
                extra_length = word_length + 1 if line else word_length
                text = f'{text} {word}' if text else word
                text_length += extra_length
            else:
                print(text)
                text = ''
                text_length = 0

        if text:
            print(text)

In [14]:
# 1) Input question into LLM agent and obtain response.
question = 'Can you explain what BTTC or repo `bt_test_common` is?'
resp = llm_agent.answer(question)

In [15]:
print_answer(resp)

I'm not familiar with a specific technology or repository called "BTTC" or "bt_test_common." It's
that it could be a project-specific acronym or a custom repository created by a particular
or team. If you can provide more context or details about what BTTC stands for or where you
it, I may be able to offer more insights or assistance.


In [16]:
question = "How to turn off the bluetooth of device by package `bttc`?"
resp = llm_agent.answer(question)

In [17]:
print_answer(resp)

I'm not familiar with a package called `bttc` for turning off Bluetooth on a device. However, if you
looking to programmatically turn off Bluetooth on a device using a software package or library, you
need to use platform-specific APIs or libraries.

For example, on Android, you can use the BluetoothAdapter class to enable or disable Bluetooth.
an example code snippet in Java:

```java
BluetoothAdapter bluetoothAdapter = BluetoothAdapter.getDefaultAdapter();
if (bluetoothAdapter != null) {
if (bluetoothAdapter.isEnabled()) {
bluetoothAdapter.disable();
}
}
```

On iOS, you can use the CoreBluetooth framework to manage Bluetooth functionality. Here's an example
snippet in Swift:

```swift
import CoreBluetooth

let centralManager = CBCentralManager()
if centralManager.state == .poweredOn {
centralManager.stopScan()
centralManager.cancelPeripheralConnection(peripheral)
}
```

If you provide more information about the specific platform or programming language you are using, I
provide more 

In this block, you import `os`, `json`, and `openai` and set the <b><font color='orange'>TOKENIZERS_PARALLELISM</font></b> environment variable to "false". Setting this environment variable to "false" will suppress a warning related to [**huggingface tokenizers**](https://huggingface.co/docs/transformers/main_classes/tokenizer). You then load the JSON object that stores your OpenAI API key.

The context message, `You are a software engineer.`, helps set the behavior of the LLM so that its responses are more likely to have a desired tone. This type of message is also sometimes called a [**role prompt**](https://realpython.com/practical-prompt-engineering/#add-a-role-prompt-to-set-the-tone). <b>The user message, `How to turn off the bluetooth of device by package `bttc`?`, is the actual question or task that you want the LLM to respond to.</b>

<a id='sect1_4'></a>
### <b><font color='darkgreen'>Provide Context to the LLM</font></b> ([back](#sect1))
<b><font size='3ptx'>As you can see, the LLM gives you a fairly generic description of what it takes to promote customer satisfaction. None of this information is particularly useful to you because it isn’t specific to repo `bttc`.</font></b>

To make this response more tailored to your business, you need to provide the LLM with some reviews as context:

In [18]:
RAG_CONTEXT_FORMAT = """
You are a software engineer with less experience in using bttc.
Use the following collected information to answer questions: {}
"""

RAG_QUESTION_FORMAT = """
{}

Please give a short description coming after with the bullet point list as summary.
"""

# Setup RAG settings.
llm_agent.rag_context_format = RAG_CONTEXT_FORMAT
llm_agent_rag_question_format = RAG_QUESTION_FORMAT

In [19]:
question1 = 'Can you explain what BTTC or repo `bt_test_common` is?'

related_docs = collection.query(
    query_texts=[question1],
    n_results=10,
    include=["documents", "metadatas"])

In [20]:
doc_str = related_docs["documents"][0][0]
doc_metadata = related_docs["metadatas"][0][0]
print(doc_metadata)
print(doc_str[:500])

{'file_path': '/usr/local/google/home/johnkclee/Github/bt_test_common/README.md', 'file_type': 'md'}
## Common utilities used in BT testing
This package is used to hold common utilities for BT testing.

## Installation
You can install the released package from pip:

```shell
$ pip install bttc
```

or download the source then run command below to use the latest version:

```shell
$ git clone https://github.com/johnklee/bt_test_common.git
$ cd bt_test_common
$ pip install .
```

You may need `sudo` for the above commands if your system has certain permission restrictions.

## Basic Usage

### Re


In [21]:
docs_str = ",".join(related_docs["documents"][0])
RAG_CONTEXT_FORMAT.format(docs_str)[:20]

'\nYou are a software '

We setup `rag_max_doc_count=5` to avoid below exception:
```python
BadRequestError: Error code: 400 - {
  'error': {
      'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 16392 tokens....",
      'type': 'invalid_request_error',
      'param': 'messages', 'code': 'context_length_exceeded'}}
```

In [22]:
full_resp = llm_agent.answer_with_rag(
    collection=collection,  # Vectorstore containing BTTC embeddings
    question=question1,     # Question to query the RAG-augmented LLM
    rag_max_doc_count=5,    # Maximum number of relevant embeddings to consider
    return_full=True,       # Returns full information including composed question and context
)

In [23]:
print_answer(full_resp.response)

BTTC stands for Bluetooth Test Common (BTTC), which is a package used for testing Bluetooth
It provides common utilities and tools for Bluetooth testing purposes. The `bt_test_common`
contains modules and functions related to Bluetooth testing, such as `bt_utils`, `ble_utils`, and

The `bt_test_common` repository includes unit test cases for these modules to ensure the
and correctness of the Bluetooth testing utilities provided by BTTC. The unit tests cover scenarios
checking Bluetooth device states, handling device configurations, scanning for devices, and more.

Overall, BTTC and the `bt_test_common` repository are resources for software engineers and
working on Bluetooth testing to streamline their testing processes and ensure the reliability of
functionalities in their applications.


In [24]:
full_resp.question

'Can you explain what BTTC or repo `bt_test_common` is?'

In [26]:
print(full_resp.context[:500])


You are a software engineer with less experience in using bttc.
Use the following collected information to answer questions: ## Common utilities used in BT testing
This package is used to hold common utilities for BT testing.

## Installation
You can install the released package from pip:

```shell
$ pip install bttc
```

or download the source then run command below to use the latest version:

```shell
$ git clone https://github.com/johnklee/bt_test_common.git
$ cd bt_test_common
$ pip install


Then for the second question:

In [27]:
question2 = "How to turn off the bluetooth of device by using package `bttc`?"
resp = llm_agent.answer_with_rag(
    collection=collection,  # Vectorstore containing BTTC embeddings
    question=question2,     # Question to query the RAG-augmented LLM
    rag_max_doc_count=5,    # Maximum number of relevant embeddings to consider
    return_full=True,       # Returns full information including composed question and context
)

In [28]:
print(resp.context)


You are a software engineer with less experience in using bttc.
Use the following collected information to answer questions: ## Preface
Thei `bt_utils` module provides utility functions for common Bluetooth
operations, such as enabling/disabling Bluetooth and retrieving a list of
paired devices.

## Usage
The `bttc.bt_utils.BTModule` class is loaded automatically when you retrieve a
device using the `bttc.get()` function.  For example:
```python
# Import the package `bttc`
>>> import bttc

# Retrieve your device. Please replace '36121FDJG000GR' with your device's
# serial number.
>>> dut = bttc.get('36121FDJG000GR')
```
You can access supported Bluetooth operations through the device's `bt`
attribute.

### Get Bluetooth status
You can call method `dut.bt.is_enabled` or access attribute `dut.bt.enabled` to
get the Bluetooth status of device. e.g.:
```python
>>> dut.bt.is_enabled()   # Checks if Bluetooth is enabled.
False
>>> dut.bt.enable()       # Enable the Bluetooth of device.
>>> 

In [29]:
print_answer(resp.response)

To turn off the Bluetooth of a device using the `bttc` package, you can follow these steps:

1. Retrieve your device using the `bttc.get()` function.
2. Access the Bluetooth attribute of the device.
3. Call the `disable()` method to turn off the Bluetooth.

Here is an example code snippet to demonstrate how to turn off the Bluetooth of a device using the
package:

```python
# Import the package `bttc`
import bttc

# Retrieve your device. Please replace '36121FDJG000GR' with your device's serial number.
dut = bttc.get('36121FDJG000GR')

# Disable the Bluetooth of the device
dut.bt.disable()
```

By calling `dut.bt.disable()`, you will turn off the Bluetooth of the device.


In [30]:
print_answer(llm_agent.answer_with_rag(
    collection=collection,                                  # Vectorstore containing BTTC embeddings
    question='How to get the MAC address of device by bttc?',     # Question to query the RAG-augmented LLM
    rag_max_doc_count=5,                                    # Maximum number of relevant embeddings to consider
))

To get the MAC address of a device using the `bttc` package, you can follow these steps:

1. Import the necessary modules:
```python
from bttc import bt_utils
```

2. Retrieve your device by its serial number:
```python
dut = bt_utils.bind('36121FDJG000GR')
```

3. Get the Bluetooth MAC address of the device:
```python
mac_address = dut.bt.get_bluetooth_mac_address()
print(mac_address)
```

This code snippet will retrieve the Bluetooth MAC address of the device with the serial number
using the `bttc` package.


As before, you import dependencies, define configuration variables, set your OpenAI API key, and obtain the collection `bttc_docs`. You then define context and question variables that you’ll feed into an LLM for inference. <b>The key difference in context is the `{}` at the end, which will be replaced with relevant reviews that give the LLM context to base its answers on</b>.

You then pass the question into <font color='blue'>collection.query()</font> and request ten documents that are most related to the question. In this query. Lastly, you pass the comma-separated `review_str` into context and request an answer from "`gpt-3.5-turbo`".

Notice how much more specific and detailed ChatGPT’s response is now that you’ve given it relevant documents as context. For example, if you look through the documents in `related_docs`, then you’ll see source code that mention how to turn off bluetooth which are incorporated into the LLM’s response.

<b><font color='darkred'>Note.</font></b>
> <b>It’s a common misconception that setting <font color='blue'>temperature=0</font> guarantees deterministic responses from ChatGPT</b>. While responses are closer to deterministic when temperature=0, [there’s no guarantee](https://arxiv.org/pdf/2308.02828#:~:text=We%20study%20the%20influence%20of,contrary%20to%20many%20people%27s%20beliefs.) that you’ll get the same response for identical requests. Because of this, ChatGPT might output slightly different results than what you see in this example.

<b>You’ve now seen why vector databases like ChromaDB are so useful for adding context to LLMs. In this example, you’ve scratched the surface of what you can create with ChromaDB, so just think about all the potential use cases for applications like this</b>. The LLM and vector database landscape will likely continue to evolve at a rapid pace, but you can now feel confident in your understanding of how the two technologies interplay with each other.

## <b><font color='darkblue'>Supplement</font></b>
* [Medium - Code Generation using Retrieval Augmented Generation + LangChain](https://medium.com/@rubenszimbres/code-generation-using-retrieval-augmented-generation-langchain-861e3c1a1a53)
* [Chroma API Cheetsheet](https://docs.trychroma.com/api-reference)
* [RealPython - Prompt Engineering: A Practical Example](https://realpython.com/practical-prompt-engineering/#add-a-role-prompt-to-set-the-tone)